# Installing the required **Libraries**

In [ ]:
!pip install -U torch torchvision
!pip install git+https://github.com/facebookresearch/fvcore.git
!git clone https://github.com/facebookresearch/detectron2 detectron2_repo
!pip install -e detectron2_repo

# Connecting Colab to Drive


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Getting the Data

In [ ]:
!cp /content/drive/MyDrive/new_data_aug.zip /content

Unzip the data

In [ ]:
!unzip /content/new_data_aug.zip

In [ ]:
%cd /content/detectron2_repo

Importing the libraries

In [ ]:
import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()

# import some common libraries
import numpy as np
import os, json, cv2, random
from google.colab.patches import cv2_imshow

# import some common detectron2 utilities
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog, DatasetCatalog

Registering the Training and Validation images as coco instances

---



In [ ]:
from detectron2.data.datasets import register_coco_instances
# register_coco_instances("glass_crack", {}, "/content/oliena-SingleDefectMask-updated/train.json", "/content/oliena-SingleDefectMask-updated/train/images")

register_coco_instances("water_train", {}, "/content/new_data_aug/train/images/train.json", "/content/new_data_aug/train/images")
register_coco_instances("water_valid", {}, "/content/new_data_aug/valid/images/valid.json", "/content/new_data_aug/valid/images")

In [ ]:
fruits_nuts_metadata_train = MetadataCatalog.get("water_train")
fruits_nuts_metadata_valid = MetadataCatalog.get("water_valid")
print(fruits_nuts_metadata_train)

In [ ]:
dataset_dicts_train = DatasetCatalog.get("water_train")
dataset_dicts_valid = DatasetCatalog.get("water_valid")

In [ ]:

import random
from detectron2.utils.visualizer import Visualizer
for d in random.sample(dataset_dicts_train, 3):
    print(d)
    img = cv2.imread(d["file_name"])
    visualizer = Visualizer(img[:, :, ::-1], metadata=fruits_nuts_metadata_train, scale=0.5)
    vis = visualizer.draw_dataset_dict(d)
    cv2_imshow(vis.get_image()[:, :, ::-1])

Customize configurations from model zoo




In [ ]:
from detectron2.engine import DefaultTrainer
from detectron2.config import get_cfg
import os
cfg = get_cfg()
cfg.merge_from_file(
    "/content/detectron2_repo/configs/COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml"
)
cfg.DATASETS.TRAIN = ("water_train",)
cfg.DATASETS.TEST = ("water_valid",)  # no metrics implemented for this dataset
cfg.DATALOADER.NUM_WORKERS = 2
cfg.MODEL.WEIGHTS = "detectron2://COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x/137849600/model_final_f10217.pkl"  # initialize from model zoo
cfg.SOLVER.IMS_PER_BATCH = 2
cfg.SOLVER.BASE_LR = 0.002
cfg.SOLVER.MAX_ITER = (
    1000
)  # 300 iterations seems good enough, but you can certainly train longer
cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = (
    128
)  # faster, and good enough for this toy dataset
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 1  # 3 classes (data, fig, hazelnut)

os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)
trainer = DefaultTrainer(cfg)


Training on Custom Dataset

In [ ]:
#trainer.resume_or_load(resume=False)
#trainer.train()

In [ ]:
from detectron2.engine import DefaultTrainer
from detectron2.config import get_cfg
import os
cfg.MODEL.WEIGHTS = os.path.join("/content/drive/MyDrive/model_final_new_data_aug.pth")
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.8   # set the testing threshold for this model
cfg.DATASETS.TEST = ("water_test", )
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 1
predictor = DefaultPredictor(cfg)

In [ ]:

from detectron2.utils.visualizer import ColorMode

for d in random.sample(dataset_dicts_valid, 3):    
    im = cv2.imread(d["file_name"])
    outputs = predictor(im)
    v = Visualizer(im[:, :, ::-1],
                   metadata=fruits_nuts_metadata_valid, 
                   scale=0.8, 
                   instance_mode=ColorMode.IMAGE_BW   # remove the colors of unsegmented pixels
    )
    v = v.draw_instance_predictions(outputs["instances"].to("cpu"))
    cv2_imshow(v.get_image()[:, :, ::-1])

In [ ]:
Inference on an image

In [ ]:

from detectron2.utils.visualizer import ColorMode

#for d in random.sample(dataset_dicts_valid, 3):    
im = cv2.imread("/Capture4.JPG")
im = cv2.resize(im,[512,512])
outputs = predictor(im)
masks=outputs["instances"].pred_masks
v = Visualizer(im[:, :, ::-1],
                metadata=fruits_nuts_metadata_valid, 
                scale=0.8, 
                instance_mode=ColorMode.IMAGE_BW   # remove the colors of unsegmented pixels
)
v = v.draw_instance_predictions(outputs["instances"].to("cpu"))

x = []
for mask in masks:
    x.append(mask.sum().item())  

cv2_imshow(v.get_image()[:, :, ::-1])

In [ ]:
!rm -rf /content/frames
!rm -rf /content/output_frames


In [ ]:
import os
os.makedirs('/content/frames')

In [ ]:
import os
os.mkdir('/content/output_frames')

Inference on the video 

In [ ]:
import cv2

cap = cv2.VideoCapture(r'/content/drive/MyDrive/clip2.mp4')

fps = int(cap.get(cv2.CAP_PROP_FPS))
print(fps)
frame_rate = cap.get(5)
print("frame rate",frame_rate)
save_interval = 1
count = 0
frame_count = 0
while cap.isOpened():
  ret, frame = cap.read()
  if ret:
    frame_count += 1
    if frame_count % (fps * save_interval) == 0:
      count = count + 1
      frame = cv2.resize(frame,[256,256])
      outputs = predictor(frame)

      masks=outputs["instances"].pred_masks
     
      x = []
      for mask in masks:
            x.append((mask.sum()).item())
      print(x)
      v = Visualizer(frame[:, :, ::-1],metadata=fruits_nuts_metadata_valid,scale=1.2)
      v = v.draw_instance_predictions(outputs["instances"].to("cpu"))
      output_frame = v.get_image()[:, :, ::-1]
      a=mask.sum() 
      img=output_frame.copy()
      output_frame = cv2.putText(img, text='Size of water mask: '+str(x), org=(50, 50), fontFace=cv2.FONT_HERSHEY_TRIPLEX, fontScale=0.75, color=(0,0,0),thickness=1)
      
      cv2.imwrite('/content/output_frames/'+str(count)+'water.jpg',output_frame)
      
      frame_count = 0

# Break the loop
  else:
    break

cap.release()
cv2.destroyAllWindows()

In [ ]:
!zip -r /content/water_frames2.zip /content/output_frames

Postprocess the model output to extract the predicted masks.

In [ ]:
import os
import natsort

dir_path = "/content/output_frames"

filenames = os.listdir(dir_path)
sorted_filenames = natsort.natsorted(filenames)

print(sorted_filenames)


Run inference on the input data using the loaded model.

In [ ]:
import numpy as np
import glob
import os
img_array = []
filename = []
import cv2
dir_path = "/content/output_frames"

filenames = os.listdir(dir_path)
for filename in sorted_filenames:
    filename = '/content/output_frames/'+filename
    print(filename)
    
    img = cv2.imread(filename)
    height, width, layers = img.shape
    size = (width,height)
    img_array.append(img)
    
out = cv2.VideoWriter('/content/project8.avi',cv2.VideoWriter_fourcc(*'DIVX'), 1, size)
 
for i in range(len(img_array)):
    #print(i)
    out.write(img_array[i])
out.release()  